In [1]:
import ROOT

Welcome to JupyROOT 6.20/04


In [2]:
import numpy as np

In [3]:
!wget -U 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.1.6) Gecko/20070802 SeaMonkey/1.1.4' https://arxiv.org/src/1411.4312v6/anc/lester_mt2_bisect.h
ROOT.gInterpreter.ProcessLine('#include "lester_mt2_bisect.h"')
myObj=ROOT.asymm_mt2_lester_bisect()
def calcMt2(V1,V2,t_sig):
    Met_X =(t_sig.met_et*np.cos(t_sig.met_phi))
    Met_Y = (t_sig.met_et*np.sin(t_sig.met_phi))
    return myObj.get_mT2(V1.M(),V1.Px(),V1.Py(),V2.M(),V2.Px(),V2.Py(),Met_X,Met_Y,700,700) 


--2020-05-09 03:50:31--  https://arxiv.org/src/1411.4312v6/anc/lester_mt2_bisect.h
Resolving arxiv.org (arxiv.org)... 128.84.21.199
Connecting to arxiv.org (arxiv.org)|128.84.21.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘lester_mt2_bisect.h.3’

lester_mt2_bisect.h     [ <=>                ]  24.96K  --.-KB/s    in 0.04s   

2020-05-09 03:50:32 (672 KB/s) - ‘lester_mt2_bisect.h.3’ saved [25562]



In [4]:
def filter(t_sig,h_sig,h2_sig):
    n = 0
    mllfail = 0
    loosecut = 0
    tightcut = 0
    btagged = 0.6459 ### 0.6459 taken from paper sent by Andeen to get 77% effeciency of b-tagged jets using MV2c10
    Mllfail = 0
    Mt2fail = 0
    print("Filtering starting now")
    for event in t_sig:
        n += 1
        if(n%10000==0):
            print("Event ", n ,"out of ", t_sig.GetEntries())
        ##triggered by single electron or muon
        if(t_sig.trigE == True or t_sig.trigM == True): 
        
            ## exactly 2 SFOS leptons
            if(t_sig.lep_n == 2 and t_sig.lep_type[1] == t_sig.lep_type[0] and t_sig.lep_charge[0]+t_sig.lep_charge[1]==0): 
            
                #check energy of electrons and muons
                if(t_sig.lep_type[0] == 11 and t_sig.lep_pt[0] > 25) or (t_sig.lep_type[0] == 13 and t_sig.lep_pt[0] > 20): 
                
                    ##look at b-tagged and non-b-tagged jets
                    ###########NOT BEING USED###########################
                    l = 0
                    for i in range(len(t_sig.jet_MV2c10)):
                        if(t_sig.jet_MV2c10[i] > btagged and t_sig.jet_pt[i] > 20): 
                            l = 1
                            
                        if(t_sig.jet_MV2c10[i] < btagged and t_sig.jet_pt[i] > 60):
                            l = 1
                        
                    if(l==0):
                    
                        ########################MT2 CALC#########################                   
                        lep0 = ROOT.Math.PtEtaPhiEVector(t_sig.lep_pt[0],t_sig.lep_eta[0],t_sig.lep_phi[0],t_sig.lep_E[0])
                        lep1 = ROOT.Math.PtEtaPhiEVector(t_sig.lep_pt[1],t_sig.lep_eta[1],t_sig.lep_phi[1],t_sig.lep_E[1])
                        Sum = lep1 + lep0
                        mll = Sum.M()
                    
                        Mt2 = calcMt2(lep0,lep1,t_sig)   
                        #########loose fit#######
                        if(mll > 111*10**3 and Mt2 > 100*10**3):
                            h_sig.Fill((t_sig.met_et)/1000.)
                            loosecut += 1
                    
                
                            #######tight fit##########
                            if(mll > 300*10**3 and Mt2 > 130*10**3):
                                h2_sig.Fill((t_sig.met_et)/1000.)
                                tightcut +=1
                            else:
                                if(mll < 300*10**3):
                                    Mllfail += 1
                                if(Mt2 < 130*10**3):
                                    Mt2fail += 1
                        else:
                            mllfail += 1
    print(loosecut, "Event's passed loose cut")
    print(tightcut, "Event's passed tight cut")
    print(mllfail, "Event's failed loose cut bc either Mll or Mt2 is not large enough")
    print(Mllfail, "Event's failed tight cut bc mll is not large enough")
    print(Mt2fail, "Event's failed tight cut bc Mt2 is not large enough")
    print("       ")
    print("Done!")

Upload MC Simulations

In [5]:
## Reads data from MC simulation of slepton creation
sig = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_392999.SlepSlep_direct_700p0_300p0_2L8.2lep.root")
t_sig = sig.Get("mini")
print(t_sig.GetEntries(), " Entries for signal")

h_sig = ROOT.TH1F("h_sig","Loose Cut",20,0,2000)
h2_sig = ROOT.TH1F("h2_sig","Tight Cut",20,0,2000)

6898  Entries for signal


In [6]:
filter(t_sig,h_sig,h2_sig)

Filtering starting now
1876 Event's passed loose cut
1423 Event's passed tight cut
533 Event's failed loose cut bc either Mll or Mt2 is not large enough
343 Event's failed tight cut bc mll is not large enough
123 Event's failed tight cut bc Mt2 is not large enough
       
Done!


#=========================================================
# To disable this message, place a call to 
# 
#     asymm_mt2_lester_bisect::disableCopyrightMessage();
# 
# somewhere before you begin to calculate your MT2 values.
#=========================================================
# You are calculating symmetric or asymmetric MT2 using
# the implementation defined in:
# 
#     http://arxiv.org/abs/1411.4312
# 
# Please cite the paper above if you use the MT2 values
# for a scholarly purpose. For the variable MT2 itself,
# please also cite:
# 
#     http://arxiv.org/abs/hep-ph/9906349
#=========================================================




BackGround Data

In [7]:
## DiBoson process ZZ -> qqll
bkg1 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363356.ZqqZll.2lep.root")
t_bkg1 = bkg1.Get("mini")
print(t_bkg1.GetEntries()," Entries for background")

# DiBoson process WZ -> qqll
bkg2 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363358.WqqZll.2lep.root")
t_bkg2 = bkg2.Get("mini")
print(t_bkg2.GetEntries()," Entries for background")

# DiBoson process WZ -> llvv
bkg3 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363492.llvv.2lep.root")
t_bkg3 = bkg3.Get("mini")
print(t_bkg3.GetEntries()," Entries for background")

# DiBoson process w/ final state llll
bkg4 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363490.llll.2lep.root")
t_bkg4 = bkg4.Get("mini")
print(t_bkg4.GetEntries()," Entries for background")

# DiBoson process w/ final state lllv
bkg5 = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363491.lllv.2lep.root")
t_bkg5 = bkg5.Get("mini")
print(t_bkg5.GetEntries()," Entries for background")

h_bkg = ROOT.TH1F("h_bgs","Loose Cut",20,0,2000)
h2_bkg = ROOT.TH1F("h2_bgs","Tight Cut",20,0,2000)

1403146  Entries for background
1316619  Entries for background
3409043  Entries for background
4266124  Entries for background
3791795  Entries for background


In [8]:
filter(t_bkg1,h_bkg,h2_bkg)

Filtering starting now
Event  10000 out of  1403146
Event  20000 out of  1403146
Event  30000 out of  1403146
Event  40000 out of  1403146
Event  50000 out of  1403146
Event  60000 out of  1403146
Event  70000 out of  1403146
Event  80000 out of  1403146
Event  90000 out of  1403146
Event  100000 out of  1403146
Event  110000 out of  1403146
Event  120000 out of  1403146
Event  130000 out of  1403146
Event  140000 out of  1403146
Event  150000 out of  1403146
Event  160000 out of  1403146
Event  170000 out of  1403146
Event  180000 out of  1403146
Event  190000 out of  1403146
Event  200000 out of  1403146
Event  210000 out of  1403146
Event  220000 out of  1403146
Event  230000 out of  1403146
Event  240000 out of  1403146
Event  250000 out of  1403146
Event  260000 out of  1403146
Event  270000 out of  1403146
Event  280000 out of  1403146
Event  290000 out of  1403146
Event  300000 out of  1403146
Event  310000 out of  1403146
Event  320000 out of  1403146
Event  330000 out of  1403

In [9]:
filter(t_bkg2,h_bkg,h2_bkg)

Filtering starting now
Event  10000 out of  1316619
Event  20000 out of  1316619
Event  30000 out of  1316619
Event  40000 out of  1316619
Event  50000 out of  1316619
Event  60000 out of  1316619
Event  70000 out of  1316619
Event  80000 out of  1316619
Event  90000 out of  1316619
Event  100000 out of  1316619
Event  110000 out of  1316619
Event  120000 out of  1316619
Event  130000 out of  1316619
Event  140000 out of  1316619
Event  150000 out of  1316619
Event  160000 out of  1316619
Event  170000 out of  1316619
Event  180000 out of  1316619
Event  190000 out of  1316619
Event  200000 out of  1316619
Event  210000 out of  1316619
Event  220000 out of  1316619
Event  230000 out of  1316619
Event  240000 out of  1316619
Event  250000 out of  1316619
Event  260000 out of  1316619
Event  270000 out of  1316619
Event  280000 out of  1316619
Event  290000 out of  1316619
Event  300000 out of  1316619
Event  310000 out of  1316619
Event  320000 out of  1316619
Event  330000 out of  1316

In [10]:
filter(t_bkg3,h_bkg,h2_bkg)

Filtering starting now
Event  10000 out of  3409043
Event  20000 out of  3409043
Event  30000 out of  3409043
Event  40000 out of  3409043
Event  50000 out of  3409043
Event  60000 out of  3409043
Event  70000 out of  3409043
Event  80000 out of  3409043
Event  90000 out of  3409043
Event  100000 out of  3409043
Event  110000 out of  3409043
Event  120000 out of  3409043
Event  130000 out of  3409043
Event  140000 out of  3409043
Event  150000 out of  3409043
Event  160000 out of  3409043
Event  170000 out of  3409043
Event  180000 out of  3409043
Event  190000 out of  3409043
Event  200000 out of  3409043
Event  210000 out of  3409043
Event  220000 out of  3409043
Event  230000 out of  3409043
Event  240000 out of  3409043
Event  250000 out of  3409043
Event  260000 out of  3409043
Event  270000 out of  3409043
Event  280000 out of  3409043
Event  290000 out of  3409043
Event  300000 out of  3409043
Event  310000 out of  3409043
Event  320000 out of  3409043
Event  330000 out of  3409

Event  2690000 out of  3409043
Event  2700000 out of  3409043
Event  2710000 out of  3409043
Event  2720000 out of  3409043
Event  2730000 out of  3409043
Event  2740000 out of  3409043
Event  2750000 out of  3409043
Event  2760000 out of  3409043
Event  2770000 out of  3409043
Event  2780000 out of  3409043
Event  2790000 out of  3409043
Event  2800000 out of  3409043
Event  2810000 out of  3409043
Event  2820000 out of  3409043
Event  2830000 out of  3409043
Event  2840000 out of  3409043
Event  2850000 out of  3409043
Event  2860000 out of  3409043
Event  2870000 out of  3409043
Event  2880000 out of  3409043
Event  2890000 out of  3409043
Event  2900000 out of  3409043
Event  2910000 out of  3409043
Event  2920000 out of  3409043
Event  2930000 out of  3409043
Event  2940000 out of  3409043
Event  2950000 out of  3409043
Event  2960000 out of  3409043
Event  2970000 out of  3409043
Event  2980000 out of  3409043
Event  2990000 out of  3409043
Event  3000000 out of  3409043
Event  3

In [11]:
filter(t_bkg4,h_bkg,h2_bkg)

Filtering starting now
Event  10000 out of  4266124
Event  20000 out of  4266124
Event  30000 out of  4266124
Event  40000 out of  4266124
Event  50000 out of  4266124
Event  60000 out of  4266124
Event  70000 out of  4266124
Event  80000 out of  4266124
Event  90000 out of  4266124
Event  100000 out of  4266124
Event  110000 out of  4266124
Event  120000 out of  4266124
Event  130000 out of  4266124
Event  140000 out of  4266124
Event  150000 out of  4266124
Event  160000 out of  4266124
Event  170000 out of  4266124
Event  180000 out of  4266124
Event  190000 out of  4266124
Event  200000 out of  4266124
Event  210000 out of  4266124
Event  220000 out of  4266124
Event  230000 out of  4266124
Event  240000 out of  4266124
Event  250000 out of  4266124
Event  260000 out of  4266124
Event  270000 out of  4266124
Event  280000 out of  4266124
Event  290000 out of  4266124
Event  300000 out of  4266124
Event  310000 out of  4266124
Event  320000 out of  4266124
Event  330000 out of  4266

Event  2690000 out of  4266124
Event  2700000 out of  4266124
Event  2710000 out of  4266124
Event  2720000 out of  4266124
Event  2730000 out of  4266124
Event  2740000 out of  4266124
Event  2750000 out of  4266124
Event  2760000 out of  4266124
Event  2770000 out of  4266124
Event  2780000 out of  4266124
Event  2790000 out of  4266124
Event  2800000 out of  4266124
Event  2810000 out of  4266124
Event  2820000 out of  4266124
Event  2830000 out of  4266124
Event  2840000 out of  4266124
Event  2850000 out of  4266124
Event  2860000 out of  4266124
Event  2870000 out of  4266124
Event  2880000 out of  4266124
Event  2890000 out of  4266124
Event  2900000 out of  4266124
Event  2910000 out of  4266124
Event  2920000 out of  4266124
Event  2930000 out of  4266124
Event  2940000 out of  4266124
Event  2950000 out of  4266124
Event  2960000 out of  4266124
Event  2970000 out of  4266124
Event  2980000 out of  4266124
Event  2990000 out of  4266124
Event  3000000 out of  4266124
Event  3

In [12]:
filter(t_bkg5,h_bkg,h2_bkg)

Filtering starting now
Event  10000 out of  3791795
Event  20000 out of  3791795
Event  30000 out of  3791795
Event  40000 out of  3791795
Event  50000 out of  3791795
Event  60000 out of  3791795
Event  70000 out of  3791795
Event  80000 out of  3791795
Event  90000 out of  3791795
Event  100000 out of  3791795
Event  110000 out of  3791795
Event  120000 out of  3791795
Event  130000 out of  3791795
Event  140000 out of  3791795
Event  150000 out of  3791795
Event  160000 out of  3791795
Event  170000 out of  3791795
Event  180000 out of  3791795
Event  190000 out of  3791795
Event  200000 out of  3791795
Event  210000 out of  3791795
Event  220000 out of  3791795
Event  230000 out of  3791795
Event  240000 out of  3791795
Event  250000 out of  3791795
Event  260000 out of  3791795
Event  270000 out of  3791795
Event  280000 out of  3791795
Event  290000 out of  3791795
Event  300000 out of  3791795
Event  310000 out of  3791795
Event  320000 out of  3791795
Event  330000 out of  3791

Event  2690000 out of  3791795
Event  2700000 out of  3791795
Event  2710000 out of  3791795
Event  2720000 out of  3791795
Event  2730000 out of  3791795
Event  2740000 out of  3791795
Event  2750000 out of  3791795
Event  2760000 out of  3791795
Event  2770000 out of  3791795
Event  2780000 out of  3791795
Event  2790000 out of  3791795
Event  2800000 out of  3791795
Event  2810000 out of  3791795
Event  2820000 out of  3791795
Event  2830000 out of  3791795
Event  2840000 out of  3791795
Event  2850000 out of  3791795
Event  2860000 out of  3791795
Event  2870000 out of  3791795
Event  2880000 out of  3791795
Event  2890000 out of  3791795
Event  2900000 out of  3791795
Event  2910000 out of  3791795
Event  2920000 out of  3791795
Event  2930000 out of  3791795
Event  2940000 out of  3791795
Event  2950000 out of  3791795
Event  2960000 out of  3791795
Event  2970000 out of  3791795
Event  2980000 out of  3791795
Event  2990000 out of  3791795
Event  3000000 out of  3791795
Event  3

ATLAS DATA

In [ ]:
#Period A
ATLASfileA = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_A.2lep.root")
ATLASA = ATLASfileA.Get("mini")
print(ATLASA.GetEntries(), " Entries for signal")

#Period B
ATLASfileB = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_B.2lep.root")
ATLASB = ATLASfileB.Get("mini")
print(ATLASB.GetEntries(), " Entries for signal")

#Period C
ATLASfileC = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_C.2lep.root")
ATLASC = ATLASfileC.Get("mini")
print(ATLASC.GetEntries(), " Entries for signal")

#Period D
ATLASfileD = ROOT.TFile.Open("http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/Data/data_D.2lep.root")
ATLASD = ATLASfileD.Get("mini")
print(ATLASD.GetEntries(), " Entries for signal")

h_ATLAS = ROOT.TH1F("h_ATLAS","Loose Cut",20,0,2000)
h2_ATLAS = ROOT.TH1F("h2_ATLAS","Tight Cut",20,0,2000)

In [ ]:
filter(ATLASA,h_ATLAS,h2_ATLAS)

In [ ]:
filter(ATLASB,h_ATLAS,h2_ATLAS)

In [ ]:
filter(ATLASC,h_ATLAS,h2_ATLAS)

In [ ]:
filter(ATLASD,h_ATLAS,h2_ATLAS)

Splitting data into training and testing

Graphing Results

In [ ]:
c = ROOT.TCanvas("testCanvas","a first way to plot a variable",800,600)

In [9]:
h_bkg.SetFillStyle(3001)
h_bkg.SetFillColor(4)
h_bkg.SetLineColor(4)

h_sig.SetFillStyle(3003)
h_sig.SetFillColor(2)
h_sig.SetLineColor(2)

h_ATLAS.SetFillStyle(3001)
h_ATLAS.SetFillColor(1)
h_ATLAS.SetLineColor(1)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h_bkg,"Background (Z #rightarrow ll) ","l")
legend.AddEntry(h_sig,"Signal (Slepton Pair Creation)","l")
legend.AddEntry(h_ATLAS,"Data","l")

h_sig.SetStats(0)
h_bkg.SetStats(0)
h_ATLAS.SetStats(0)


h_ATLAS.SetMaximum(1e4)
h_ATLAS.SetMinimum(1e-1)
h_ATLAS.Draw("E1")
h_sig.Draw("histsame")
h_bkg.Draw("histsame")



legend.Draw()
c.SetLogy()
c.Draw()

h_ATLAS.GetEntries()

NameError: name 'h_ATLAS' is not defined

In [ ]:
h2_bkg.SetFillStyle(3001)
h2_bkg.SetFillColor(4)
h2_bkg.SetLineColor(4)

h2_sig.SetFillStyle(3003)
h2_sig.SetFillColor(2)
h2_sig.SetLineColor(2)

h2_ATLAS.SetFillStyle(3003)
h2_ATLAS.SetFillColor(1)
h2_ATLAS.SetLineColor(1)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h2_bkg,"Background (WW) ","l")
legend.AddEntry(h2_sig,"Signal (H #rightarrow WW)","l")
legend.AddEntry(h2_ATLAS,"Data","l")

h2_sig.SetStats(0)
h2_bkg.SetStats(0)
h2_ATLAS.SetStats(0)

h2_ATLAS.SetMaximum(1e4)
h2_ATLAS.SetMinimum(1e-1)
h2_ATLAS.Draw("E1")
h2_sig.Draw("histsame")
h2_bkg.Draw("histsame")


legend.Draw()
c.SetLogy()
c.Draw()

Final Cut

In [57]:
def FinalCut(t_sig,h3_sig):
    n = 0
    btagged = 0.6459
    Pass = 0
    Mllfail = 0
    Mt2fail = 0
    print("Filtering starting now")
    for event in t_sig:
        n += 1
        if(n%10000==0):
            print("Event ", n ,"out of ", t_sig.GetEntries())
        ##triggered by single electron or muon
        if(t_sig.trigE == True or t_sig.trigM == True): 
        
            ## exactly 2 SFOS leptons
            if(t_sig.lep_n == 2 and t_sig.lep_type[1] == t_sig.lep_type[0] and t_sig.lep_charge[0]+t_sig.lep_charge[1]==0): 
            
                #check energy of electrons and muons
                if(t_sig.lep_type[0] == 11 and t_sig.lep_pt[0] > 25) or (t_sig.lep_type[0] == 13 and t_sig.lep_pt[0] > 20): 
                
                    ##look at b-tagged and non-b-tagged jets
                    ###########NOT BEING USED###########################
                    l = 0
                    for i in range(len(t_sig.jet_MV2c10)):
                        if(t_sig.jet_MV2c10[i] > btagged and t_sig.jet_pt[i] > 20): 
                            l = 1
                            
                        if(t_sig.jet_MV2c10[i] < btagged and t_sig.jet_pt[i] > 60):
                            l = 1
                        
                    if(l==0):
                    
                        ########################MT2 CALC#########################                   
                        lep0 = ROOT.Math.PtEtaPhiEVector(t_sig.lep_pt[0],t_sig.lep_eta[0],t_sig.lep_phi[0],t_sig.lep_E[0])
                        lep1 = ROOT.Math.PtEtaPhiEVector(t_sig.lep_pt[1],t_sig.lep_eta[1],t_sig.lep_phi[1],t_sig.lep_E[1])
                        Sum = lep1 + lep0
                        mll = Sum.M()
                    
                        Mt2 = calcMt2(lep0,lep1,t_sig)   
                        #########loose fit#######
                        if(mll > 250*10**3 and Mt2 > 115*10**3):
                            h3_sig.Fill((t_sig.met_et)/1000.)
                            
                            Pass += 1
                        else:
                            if(mll < 250*10**3):
                                Mllfail += 1
                            if(Mt2 < 115*10**3):
                                Mt2fail += 1
    print(Pass, "Event's Passed")                            
    print(Mllfail, "Event's failed tight cut bc mll is not large enough")
    print(Mt2fail, "Event's failed tight cut bc Mt2 is not large enough")
    print("       ")
    print("Done!")
    

In [ ]:
h3_sig = ROOT.TH1F("h3_sig","Final Cut",20,0,2000)

In [58]:
FinalCut(t_sig,h3_sig)

Filtering starting now
1586 Event's Passed
329 Event's failed tight cut bc mll is not large enough
537 Event's failed tight cut bc Mt2 is not large enough
       
Done!


In [ ]:
h3_bkg = ROOT.TH1F("h3_bgs","Final Cut",20,0,2000)

In [52]:
FinalCut(t_bkg1,h3_bkg)

Filtering starting now
Event  10000 out of  1403146
Event  20000 out of  1403146
Event  30000 out of  1403146
Event  40000 out of  1403146
Event  50000 out of  1403146
Event  60000 out of  1403146
Event  70000 out of  1403146
Event  80000 out of  1403146
Event  90000 out of  1403146
Event  100000 out of  1403146
Event  110000 out of  1403146
Event  120000 out of  1403146
Event  130000 out of  1403146
Event  140000 out of  1403146
Event  150000 out of  1403146
Event  160000 out of  1403146
Event  170000 out of  1403146
Event  180000 out of  1403146
Event  190000 out of  1403146
Event  200000 out of  1403146
Event  210000 out of  1403146
Event  220000 out of  1403146
Event  230000 out of  1403146
Event  240000 out of  1403146
Event  250000 out of  1403146
Event  260000 out of  1403146
Event  270000 out of  1403146
Event  280000 out of  1403146
Event  290000 out of  1403146
Event  300000 out of  1403146
Event  310000 out of  1403146
Event  320000 out of  1403146
Event  330000 out of  1403

In [53]:
FinalCut(t_bkg2,h3_bkg)

Filtering starting now
Event  10000 out of  1403146
Event  20000 out of  1403146
Event  30000 out of  1403146
Event  40000 out of  1403146
Event  50000 out of  1403146
Event  60000 out of  1403146
Event  70000 out of  1403146
Event  80000 out of  1403146
Event  90000 out of  1403146
Event  100000 out of  1403146
Event  110000 out of  1403146
Event  120000 out of  1403146
Event  130000 out of  1403146
Event  140000 out of  1403146
Event  150000 out of  1403146
Event  160000 out of  1403146
Event  170000 out of  1403146
Event  180000 out of  1403146
Event  190000 out of  1403146
Event  200000 out of  1403146
Event  210000 out of  1403146
Event  220000 out of  1403146
Event  230000 out of  1403146
Event  240000 out of  1403146
Event  250000 out of  1403146
Event  260000 out of  1403146
Event  270000 out of  1403146
Event  280000 out of  1403146
Event  290000 out of  1403146
Event  300000 out of  1403146
Event  310000 out of  1403146
Event  320000 out of  1403146
Event  330000 out of  1403

In [54]:
FinalCut(t_bkg3,h3_bkg)

Filtering starting now
Event  10000 out of  1403146
Event  20000 out of  1403146
Event  30000 out of  1403146
Event  40000 out of  1403146
Event  50000 out of  1403146
Event  60000 out of  1403146
Event  70000 out of  1403146
Event  80000 out of  1403146
Event  90000 out of  1403146
Event  100000 out of  1403146
Event  110000 out of  1403146
Event  120000 out of  1403146
Event  130000 out of  1403146
Event  140000 out of  1403146
Event  150000 out of  1403146
Event  160000 out of  1403146
Event  170000 out of  1403146
Event  180000 out of  1403146
Event  190000 out of  1403146
Event  200000 out of  1403146
Event  210000 out of  1403146
Event  220000 out of  1403146
Event  230000 out of  1403146
Event  240000 out of  1403146
Event  250000 out of  1403146
Event  260000 out of  1403146
Event  270000 out of  1403146
Event  280000 out of  1403146
Event  290000 out of  1403146
Event  300000 out of  1403146
Event  310000 out of  1403146
Event  320000 out of  1403146
Event  330000 out of  1403

In [55]:
FinalCut(t_bkg4,h3_bkg)

Filtering starting now
Event  10000 out of  1403146
Event  20000 out of  1403146
Event  30000 out of  1403146
Event  40000 out of  1403146
Event  50000 out of  1403146
Event  60000 out of  1403146
Event  70000 out of  1403146
Event  80000 out of  1403146
Event  90000 out of  1403146
Event  100000 out of  1403146
Event  110000 out of  1403146
Event  120000 out of  1403146
Event  130000 out of  1403146
Event  140000 out of  1403146
Event  150000 out of  1403146
Event  160000 out of  1403146
Event  170000 out of  1403146
Event  180000 out of  1403146
Event  190000 out of  1403146
Event  200000 out of  1403146
Event  210000 out of  1403146
Event  220000 out of  1403146
Event  230000 out of  1403146
Event  240000 out of  1403146
Event  250000 out of  1403146
Event  260000 out of  1403146
Event  270000 out of  1403146
Event  280000 out of  1403146
Event  290000 out of  1403146
Event  300000 out of  1403146
Event  310000 out of  1403146
Event  320000 out of  1403146
Event  330000 out of  1403

KeyboardInterrupt: 

In [ ]:
FinalCut(t_bkg5,h3_bkg)

In [ ]:
h3_ATLAS = ROOT.TH1F("h3_ATLAS","Final Cut",20,0,2000)

In [ ]:
FinalCut(ATLASA,h3_ATLAS)

In [ ]:
FinalCut(ATLASB,h3_ATLAS)

In [ ]:
FinalCut(ATLASC,h3_ATLAS)

In [ ]:
FinalCut(ATLAD,h3_ATLAS)

In [ ]:
g = ROOT.TCanvas("FinalCanvas","Our final graph",800,600)

h3_bkg.SetFillStyle(3001)
h3_bkg.SetFillColor(4)
h3_bkg.SetLineColor(4)

h3_sig.SetFillStyle(3003)
h3_sig.SetFillColor(2)
h3_sig.SetLineColor(2)

h3_ATLAS.SetFillStyle(3003)
h3_ATLAS.SetFillColor(1)
h3_ATLAS.SetLineColor(1)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h3_bkg,"Background (WW) ","l")
legend.AddEntry(h3_sig,"Signal (H #rightarrow WW)","l")
legend.AddEntry(h3_ATLAS,"Data","l")

h3_sig.SetStats(0)
h3_bkg.SetStats(0)
h3_ATLAS.SetStats(0)

h3_ATLAS.SetMaximum(1e5)
h3_ATLAS.SetMinimum(1e-1)
h3_ATLAS.Draw("E1")
h3_sig.Draw("histsame")
h3_bkg.Draw("histsame")


legend.Draw()
g.SetLogy()
g.Draw()